In [12]:

# Clear all variables in workspace
%reset -f 

import sys


sys.path.append("C:\\Users\\s4318841\\OneDrive - The University of Queensland\\2024\\Jupyter projects\\Jupyter_21012024\\Jupyter\\")

In [13]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import re 
import seaborn as sns
import ampseq as amp
import pathlib as path


In [4]:
# import sys
# sys.path.append("C:\\Users\\s4318841\\OneDrive - The University of Queensland\\2024\\Jupyter projects\\Jupyter_21012024\\Jupyter\\")


# # print(amp.secret)

# amp.addtolist()

# C:\Users\s4318841\OneDrive - The University of Queensland\2024\Jupyter projects\Jupyter_21012024\Jupyter
# simple python file that contains the script only
# other python files will contain the functions. 

In [ ]:
import sys 
from pprint import pprint
pprint(sys.path)




In [ ]:
# !cd C:\\Users\\uqawang8\\OneDrive - The University of Queensland\\Documents\\2022\\Jupyter\\
# !cd C:\\Mphil\2022\Experiments\Flow cytometry\ACE results

os.getcwd()

# reload module after changes made
import ampseq as amp
import importlib
importlib.reload(amp)

In [17]:
# Change to project directory which contains the Qiime results 
# !cd C:\Users\s4318841\OneDrive - The University of Queensland\2024\Jupyter projects\Jupyter_21012024\Jupyter\Ace results

# Change pathname as required
# Read Qiime results and save as a dataframe. 
df= pd.read_csv(r'Ace results\level-7.csv')

# # Remove rows with a relative abundance <1%
# df_new= df_new.loc[df_new['Relative Frequency']>0.05]

# # Reorder from descending abundances 
# df_sorted= df_new.sort_values(by='Relative Frequency', ignore_index=True, ascending=False)

# df_sorted


In [ ]:

# Totalise the counts into a column ('Total')
df['Total']=df.sum(axis=1)

df



In [ ]:
len(df.columns)

# (df.iloc[:,2]/df['Total'])

# Convert the counts to relative abundance by dividing by total counts

length_req= len(df.columns)-1
i=1
while i < length_req:
    df.iloc[:,i]=(df.iloc[:,i]/df['Total']*100).round(decimals=2)
    i+=1

print(df)

################ Need to remove the Total column or else it will have issues later

# df.iloc[:,1]/df['Total']
# df.iloc[:,2]/df['Total']


In [20]:
# print(df)

# column_names= list(df.column.values)

dfwide_columns= df.iloc[:,0];

# dfwide_columns

# print(column_names)
# # print(df)

dfwide=df.set_index('index').transpose()

dfwide.reset_index(inplace=True)

dfwide.rename(columns={'index': 'Taxon'}, inplace=True)

dfwide.head()

# Filter out any taxons which have less than 1% abundance in all samples
is_signif= dfwide.iloc[:,1:6]>=1

falsecheck= dfwide[is_signif].any(axis=1)

dfwide_final=dfwide[falsecheck]

# Reset index
dfwide_final.reset_index(inplace=True)



In [ ]:
# Extract the full taxon description 
taxonlist=dfwide_final.Taxon

#create empty list of taxa
taxondict=[]

row=[];

#get the number of rows in the data frame
len(dfwide_final.index)

# Run a for loop to add every sample/row to the dictionary
i=0
while i < len(dfwide_final.index):
    amp.addtolist(taxondict,dfwide_final,i)
    i+=1


len(taxondict)
taxondict[0]='unassigned'

# taxondict.items()
# k, v = taxondict.keys(), taxondict.values()

# convert Series to dataframe
taxondf= pd.DataFrame(taxondict, columns=['Species or Genus'])
taxondf


# # convert dictionary to dataframe
# taxondf= pd.DataFrame(taxondict.items(), columns=['Species or Genus', 'Full taxa'])

# taxondf

# # Export to excel 
# taxadict_df.to_csv('taxfile1.csv')

# # # def addtolist(row):
# # #     taxa_index=df['Feature ID'][row];
# # #     taxa_bd= df['Taxon'][row].split(";");
# # #     taxadict['sample'].append(taxa_index)


# # # # # output the dataframe as a dictionary to allow for excel exporting 
# # # # dict=df.to_dict()

# # # # amp.addlist(df)
    


In [ ]:
taxondf

# Concat the dataframes to include the abbreviated taxonomy ('Species or Genus')
df_final=pd.concat([dfwide_final, taxondf], axis=1)

# Slice dataframe to only include desired data 
finalrow=len(df_final)-1

df_final=df_final.drop(finalrow)

len(df_final.columns)

df_gg= df_final.iloc[:, 2:9]

df_gg.set_index('Species or Genus', inplace=True)

# df_final=df_final.drop('Total')

# df_final=df_final.set_index('Species or Genus')

# df_gg.rename_axis('Taxons', inplace=True)
# df_gg.rename_axis('Taxons').rename_axis('Samples', axis='columns'inplace=True)

# Final dataframe 
print(df_gg.head)

# Export to excel 
df_gg.to_csv('taxafinalfile.csv')

# We can now open the exported csv file to manually modify the sample name columns



In [ ]:
# Read the the excel file back as a dataframe
df_hm= pd.read_csv('taxafinalfile.csv')

# Construct a heatmap using Python's Seaborn library
df_final_hm=df_hm.pivot_table(index='Species or Genus')

plt.figure(figsize=(40,25))

ax=sns.heatmap(df_final_hm, robust= True, cmap='Blues', annot=True)

ax.set(title='Amplicon Heatmap', 
       xlabel="Samples",
       ylabel="Taxon")

figure=ax.get_figure()
figure.savefig('Amplicon_Heatmap.png', dpi=600)
